모듈 설치 및 임포트

In [1]:
!pip install openai 
!pip install pandas
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 10.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.7/152.7 kB 4.9 MB/s eta 0:00:00


In [108]:
import random
import time
import pandas as pd
import numpy as np
import openai
import re
import xlsxwriter
from openpyxl import load_workbook

실험에 사용될 함수 정의

In [3]:
def run_typ_experiments(num_experiments, input_file, output_file = "Typicality_result.xlsx"):
    # Phase 1: 실험 전
    # OpenAI API KEY 불러오기
    with open("OPENAI_API_KEY.txt", "r") as f:
        api_key = f.read().strip()
    openai.api_key = api_key

    # 결과를 기록할 파일 및 시트 구축
    if output_file is None:
        output_file = "Typicality_result.xlsx"
    try:
        existing_file = pd.ExcelFile(output_file, engine='openpyxl')
        existing_sheets = existing_file.sheet_names
        num_sheets = len(existing_sheets)
        Results = pd.ExcelWriter(output_file, engine="openpyxl", mode="a", if_sheet_exists='replace')
        sheet_name_list = [f"EXP {i}" for i in range(num_sheets + 1, num_sheets + num_experiments + 1)]
    except FileNotFoundError:
        Results = pd.ExcelWriter(output_file, engine="openpyxl")
        sheet_name_list = [f"EXP {i}" for i in range(1, num_experiments + 1)]

    for sheet_name in sheet_name_list:
        df = pd.DataFrame(columns=["Item", "Condition", "Sentence", "Response", "Typicality"])
        df.to_excel(Results, sheet_name=sheet_name, index=False)


    # Phase 2: 실험 시작
    # 실험 문장 파일 읽기
    materials = pd.read_excel(input_file)
    num_rows, num_columns = materials.shape

    # 문장 배치 무선화
    for i in range(num_experiments):
        sentence_order = random.sample(range(num_rows), num_rows)
        responses = []
        typicalities = []
        
        # 모든 문장에 대해 시행을 수행
        for j, row in enumerate(sentence_order):
            item = materials.iloc[row, 3]
            cell_value = materials.iloc[row, num_columns - 1]
            prompt = f'''Please judge the typicality of the following expression on a scale of 1 to 7, 
            with 1 being "not typical at all" and 7 being "extremely typical" : {cell_value}'''

            response = openai.ChatCompletion.create(
                model = "gpt-3.5-turbo",
                temperature = 0.5,
                n = 1,
                max_tokens = 128,
                stop = None,
                messages = [{"role" : "system", "content" : "Your answer should contain numeric rating scale, which is the only number in your sentence."},
                {"role" : "user", "content" : prompt}])
            time.sleep(1)
            response_text = response.choices[0]['message']['content']
            responses.append(response_text)
            progress = f"EXP {i+1}/{num_experiments} Sentence {j+1}/{num_rows}"
            print(progress)

            #Typicality 추출 및 저장
            typicality = None
            match = re.search(r"\b[1-7]\b", response_text)
            if match:
                typicality = int(match.group())
            typicalities.append(typicality)

    # Phase 3: 실험 결과를 파일에 저장
        sheet_name = sheet_name_list[i]
        df = pd.DataFrame({
            "Item": materials.iloc[sentence_order, 3],
            "Condition" : materials.iloc[sentence_order, 5],
            "Sentence": materials.iloc[sentence_order, num_columns - 1],
            "Response": responses,
            "Typicality": typicalities}, columns=["Item", "Condition", "Sentence", "Response", "Typicality"])
        df.to_excel(Results, sheet_name=sheet_name, index=False)
    Results.close()
    print("Experiment completed successfully!")

In [68]:
def run_accp_experiments(num_experiments, input_file, output_file = "Acceptability_result.xlsx"):
    # Phase 1: 실험 전
    # OpenAI API KEY 불러오기
    with open("OPENAI_API_KEY.txt", "r") as f:
        api_key = f.read().strip()
    openai.api_key = api_key

    # 결과를 기록할 파일 및 시트 구축
    if output_file is None:
        output_file = "Acceptability_result.xlsx"
    try:
        existing_file = pd.ExcelFile(output_file, engine='openpyxl')
        existing_sheets = existing_file.sheet_names
        num_sheets = len(existing_sheets)
        Results = pd.ExcelWriter(output_file, engine="openpyxl", mode="a", if_sheet_exists='replace')
        sheet_name_list = [f"EXP {i}" for i in range(num_sheets + 1, num_sheets + num_experiments + 1)]
    except FileNotFoundError:
        Results = pd.ExcelWriter(output_file, engine="openpyxl")
        sheet_name_list = [f"EXP {i}" for i in range(1, num_experiments + 1)]

    for sheet_name in sheet_name_list:
        df = pd.DataFrame(columns=["Item", "Sentence_num", "Sentence", "Response", "Acceptability"])
        df.to_excel(Results, sheet_name=sheet_name, index=False)


    # Phase 2: 실험 시작
    # 실험 문장 파일 읽기
    materials = pd.read_excel(input_file)
    num_rows, num_columns = materials.shape

    # 문장 배치 무선화
    for i in range(num_experiments):
        sentence_order = random.sample(range(num_rows), num_rows)
        responses = []
        acceptabilities = []
        
        # 모든 문장에 대해 시행을 수행
        for j, row in enumerate(sentence_order):
            item = materials.iloc[row, 2]
            cell_value = materials.iloc[row, num_columns - 1]
            prompt = f'''Please judge the acceptability of the following expression on a scale of 1 to 7, 
            with 1 being "not acceptable at all" and 7 being "extremely acceptable" : {cell_value}'''

            response = openai.ChatCompletion.create(
                model = "gpt-3.5-turbo",
                temperature = 0.5,
                n = 1,
                max_tokens = 256,
                stop = None,
                messages = [{"role" : "system", "content" : "Your answer should contain numeric rating scale, which is the only number in your sentence."},
                {"role" : "user", "content" : prompt}])
            time.sleep(1)
            response_text = response.choices[0]['message']['content']
            responses.append(response_text)
            progress = f"EXP {i+1}/{num_experiments} Sentence {j+1}/{num_rows}"
            print(progress)

            #Acceptablity 추출 및 저장
            acceptablity = None
            match = re.search(r"\b[1-7]\b", response_text)
            if match:
                acceptablity = int(match.group())
            acceptabilities.append(acceptablity)

    # Phase 3: 실험 결과를 파일에 저장
        sheet_name = sheet_name_list[i]
        df = pd.DataFrame({
            "Item": materials.iloc[sentence_order, 1],
            "Sentence_num" :materials.iloc[sentence_order, 2],
            "Sentence": materials.iloc[sentence_order, num_columns - 1],
            "Response": responses,
            "Acceptability": acceptabilities}, columns=["Item", "Sentence_num", "Sentence", "Response", "Acceptability"])
        df.to_excel(Results, sheet_name=sheet_name, index=False)
    Results.close()
    print("Experiment completed successfully!")

In [123]:
def process_typ_rawdata(input_file_path, output_file_name):
    #파일 입수 및 전처리
    input_data = pd.read_excel(input_file_path, sheet_name=None)
    output_file_path = output_file_name 

    Summary = pd.ExcelWriter(output_file_path, engine='xlsxwriter')
    exp_summary = pd.DataFrame(columns=['Item', 'Condition', 'Sentence', 'Typicality'])

    for sheet_name, sheet in input_data.items():
        sheet_sorted = sheet.sort_values('Item')
        sheet_sorted.to_excel(Summary, sheet_name=sheet_name, index=False)

        # Item 별 Typicality 평균 계산
        item_summary = sheet.groupby('Item').agg({'Typicality': 'mean', 'Condition': 'first', 'Sentence': 'first'}).reset_index()
        item_summary.rename(columns={'Typicality': 'Mean_typicality'}, inplace=True)
        exp_summary = exp_summary.append(item_summary, ignore_index=True)

    # 결과 생성 및 저장
    exp_summary = exp_summary.groupby(['Item', 'Condition', 'Sentence']).agg({'Mean_typicality': 'mean'}).reset_index()
    exp_summary.to_excel(Summary, sheet_name='Experiment_summary', index=False)
    Summary.close()
    wb = load_workbook(output_file_path)
    wb.move_sheet(wb["Experiment_summary"], offset = -len(wb.sheetnames) + 1)
    wb.save(output_file_path)

def process_accp_rawdata(input_file_path, output_file_name):
    #파일 입수 및 전처리
    input_data = pd.read_excel(input_file_path, sheet_name=None)
    output_file_path = output_file_name 

    Summary = pd.ExcelWriter(output_file_path, engine='xlsxwriter')
    exp_summary = pd.DataFrame(columns=['Item', 'Sentence_num', 'Sentence', 'Acceptability'])

    for sheet_name, sheet in input_data.items():
        sheet_sorted = sheet.sort_values('Item')
        sheet_sorted.to_excel(Summary, sheet_name=sheet_name, index=False)

        # Item 별 acceptability 평균 계산
        item_summary = sheet.groupby('Item').agg({'Acceptability': 'mean', 'Sentence_num': 'first', 'Sentence': 'first'}).reset_index()
        item_summary.rename(columns={'Acceptability': 'Mean_acceptability'}, inplace=True)
        exp_summary = exp_summary.append(item_summary, ignore_index=True)

    # 결과 생성 및 저장
    exp_summary = exp_summary.groupby(['Item', 'Sentence_num', 'Sentence']).agg({'Mean_acceptability': 'mean'}).reset_index()
    exp_summary.to_excel(Summary, sheet_name='Experiment_summary', index=False)
    Summary.close()
    wb = load_workbook(output_file_path)
    wb.move_sheet(wb["Experiment_summary"], offset = -len(wb.sheetnames) + 1)
    wb.save(output_file_path)


함수 실행 부분

In [124]:
Typ_input_file = "Typicality_Materials.xlsx"
Accp_input_file = "Acceptability_Materials.xlsx"
run_typ_experiments(30, Typ_input_file)
run_accp_experiments(30, Accp_input_file)
Typ_rawdata = "Typicality_result.xlsx"
Accp_rawdata = "Acceptability_result.xlsx"
Typ_summary = "Typicality_summary.xlsx"
Accp_summary = "Acceptability_summary.xlsx"
process_typ_rawdata(Typ_rawdata,Typ_summary)
process_accp_rawdata(Accp_rawdata,Accp_summary)

<ipython-input-123-945353bf3cf1>:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  exp_summary = exp_summary.append(item_summary, ignore_index=True)
<ipython-input-123-945353bf3cf1>:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  exp_summary = exp_summary.append(item_summary, ignore_index=True)
<ipython-input-123-945353bf3cf1>:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  exp_summary = exp_summary.append(item_summary, ignore_index=True)
<ipython-input-123-945353bf3cf1>:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  exp_summary = exp_summary.append(item_summary, ignore_index=True)
<ipython-input-123-945353bf3cf1>:41: FutureWarni